# Extract Data From Scraped JSON Files


## Load Crawled Data and Build Dataframe

### Libraries

In [0]:
import os
from google.colab import drive
import json
import pandas as pd
from datetime import datetime
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter, defaultdict
import numpy as np 
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import matplotlib
import matplotlib.dates as mdates
from textblob import TextBlob

### Mounting (need to copy data to own Google Drive)

In [0]:
drive.mount('/content/gdrive')
dir = "gdrive/My Drive/10kparagraphs"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
## sentiment analysis
def sentiment_analysis(text, paragraph_mentions = None):

  if paragraph_mentions != None:

    paragraph_i = np.argmax(paragraph_mentions)
    text = text[paragraph_i]

  blob = TextBlob(str(text))
  text_polarity = blob.sentiment.polarity
  text_subjectivity = blob.sentiment.subjectivity

  return text_polarity, text_subjectivity



## word count
def get_word_counts(text, keywords):

  text = text.lower()

  corona_count = 0
  n_words = len(text.split(" "))

  ## corona mention feature
  for k in keywords:
    corona_count += text.count(str(k))

  return corona_count, n_words


## get relative word count
def get_relative_count(word_count, n_words):

    rel_word_count = 0

    if n_words >= 1:
      rel_word_count = word_count / n_words

    return rel_word_count



## get name features
def get_name_features(filename):

  ## date-SIC-FirmID.JSON
  filename_arr = filename.split("_")
  date_str = filename_arr[0]
  datetime_object = datetime.strptime(date_str, '%Y%m%d')
  SIC = filename_arr[1]
  FirmID = filename_arr[2].split(".")[0]

  return datetime_object, SIC, FirmID


## get content features
def get_content_features(json_dict, keywords):

  corona_count = 0 
  rel_corona_count = float(0.0)
  paragraph_texts = []
  complete_text = ''
  paragraph_mentions = []
  paragraph_words = []
  paragraph_polarity = float(0.0)
  paragraph_subjectivity = float(0.0)
  report_polarity = float(0.0)
  report_subjectivity = float(0.0)


  if '0' in json_dict.keys(): ## article contains data
    paragraphs = json_dict['0'].keys()

    for p in paragraphs:

      corona_mentions = 0
      text = json_dict['0'][p]

      if type(text) == str:
        paragraph_texts.append(text)
        complete_text += (" " + text)

        corona_count, n_words = get_word_counts(text, keywords)
        paragraph_mentions.append(corona_count)
        paragraph_words.append(n_words)
    
    ## total corona countd over all paragraphs
    corona_count = sum(paragraph_mentions)
    n_words = sum(paragraph_words)
    rel_corona_count = get_relative_count(corona_count, n_words)

    report_polarity, report_subjectivity = sentiment_analysis(complete_text)
    paragraph_polarity, paragraph_subjectivity = sentiment_analysis(paragraph_texts, paragraph_mentions)

  return corona_count, rel_corona_count, paragraph_mentions, paragraph_polarity, paragraph_subjectivity, report_polarity, report_subjectivity, paragraph_texts

In [0]:
df = pd.DataFrame(columns=['JSON_file','date', 'SIC', 'FirmID','corona_mentions','corona_count','rel_corona_count','raw_text'])
keywords = ["corona","covid"]

for filename in tqdm(os.listdir(dir)):
    if filename.endswith(".JSON"):
      
        json_path = os.path.join(dir, filename)
        datetime_object, SIC, FirmID = get_name_features(filename)

        with open(json_path) as f:
            json_dict = json.load(f)
            corona_count, rel_corona_count, paragraph_mentions, paragraph_polarity, paragraph_subjectivity, report_polarity, report_subjectivity, paragraph_texts = get_content_features(json_dict, keywords)

        ## append everything to Dataframe
        df = df.append({"JSON_file": filename, "date": datetime_object, "FirmID": FirmID, "SIC":  SIC, "corona_mentions": paragraph_mentions, "paragraph_sentiment": paragraph_polarity, "paragraph_subjectivity": paragraph_subjectivity, "report_sentiment": report_polarity, "report_subjectivity": report_subjectivity, "corona_count": corona_count, "rel_corona_count": round(rel_corona_count,4), "raw_text": paragraph_texts}, ignore_index=True)

    else:
        continue

100%|██████████| 1763/1763 [10:34<00:00,  2.78it/s]


In [0]:
df.to_pickle("gdrive/My Drive/k_10_corona.pkl")
df_no_text = df.drop(columns=["raw_text"])
df_no_text.to_csv("gdrive/My Drive/k_10_corona.csv", encoding = 'utf-8', index=False)